In [4]:
import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import joblib

In [5]:
# Download data for training
currency_data = yf.download('USDINR=X', start='2020-01-01', end='2023-01-01')
currency_data = currency_data[['Close']].dropna()


[*********************100%***********************]  1 of 1 completed


In [6]:
# Objective function for optimization (Mean Squared Error)
def objective_function(hyperparameters, data):
    lstm_units = int(hyperparameters[0])  # Number of LSTM units
    learning_rate = hyperparameters[1]    # Learning rate
    batch_size = int(hyperparameters[2])  # Batch size

    # Data preprocessing
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data.values)

    # Prepare the data for LSTM
    def create_dataset(data, time_step=60):
        X, y = [], []
        for i in range(time_step, len(data)):
            X.append(data[i-time_step:i, 0])  # past 'time_step' days
            y.append(data[i, 0])  # next day price
        return np.array(X), np.array(y)

    X, y = create_dataset(scaled_data)

    # Reshape data into 3D input for LSTM
    X = X.reshape(X.shape[0], X.shape[1], 1)

    # Build and compile the LSTM model
    model = Sequential()
    model.add(tf.keras.layers.Input(shape=(X.shape[1], 1)))  # Proper input layer
    model.add(LSTM(units=lstm_units, return_sequences=True))
    model.add(LSTM(units=lstm_units, return_sequences=False))
    model.add(Dense(units=1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mean_squared_error')

    # Train the model
    model.fit(X, y, epochs=10, batch_size=batch_size, verbose=0)

    # Evaluate the model
    mse = model.evaluate(X, y, verbose=0)
    return mse

In [7]:
# Define the HOA algorithm with constraints on hyperparameters
def hoa_algorithm(data, population_size=10, max_iter=100, search_space=(-1, 1)):
    # Initialize population with constrained ranges
    population = np.random.uniform(search_space[0], search_space[1], (population_size, 3))  # Hyperparameters: LSTM units, learning rate, batch size
    
    # Enforce constraints
    population[:, 0] = np.clip(population[:, 0], 10, 200)  # LSTM units between 10 and 200
    population[:, 2] = np.clip(population[:, 2], 16, 64)   # Batch size between 16 and 64
    population[:, 1] = np.clip(population[:, 1], 0.0001, 0.01)  # Learning rate between 0.0001 and 0.01
    
    best_solution = population[0]
    best_score = objective_function(best_solution, data)
    
    for iteration in range(max_iter):
        for i in range(population_size):
            # Small perturbations for hyperparameters
            candidate = population[i] + np.random.uniform(-0.1, 0.1, 3)
            
            # Apply constraints to candidate solution
            candidate[0] = np.clip(candidate[0], 10, 200)  # LSTM units
            candidate[2] = np.clip(candidate[2], 16, 64)   # Batch size
            candidate[1] = np.clip(candidate[1], 0.0001, 0.01)  # Learning rate
            
            candidate_score = objective_function(candidate, data)
            
            if candidate_score < best_score:
                best_solution = candidate
                best_score = candidate_score
        
        print(f"Iteration {iteration+1}/{max_iter} - Best Score: {best_score}")
    
    return best_solution, best_score


In [8]:
# Run HOA to optimize hyperparameters
best_hyperparameters, best_mse = hoa_algorithm(currency_data)
print(f"Best Hyperparameters: {best_hyperparameters}, with MSE: {best_mse}")

Iteration 1/100 - Best Score: 0.0008870054152794182
Iteration 2/100 - Best Score: 0.0008870054152794182
Iteration 3/100 - Best Score: 0.0008870054152794182
Iteration 4/100 - Best Score: 0.0008870054152794182
Iteration 5/100 - Best Score: 0.0008384888642467558
Iteration 6/100 - Best Score: 0.0008384888642467558
Iteration 7/100 - Best Score: 0.0008384888642467558
Iteration 8/100 - Best Score: 0.0008384888642467558
Iteration 9/100 - Best Score: 0.0008384888642467558
Iteration 10/100 - Best Score: 0.0008384888642467558
Iteration 11/100 - Best Score: 0.0008384888642467558
Iteration 12/100 - Best Score: 0.000763254938647151
Iteration 13/100 - Best Score: 0.000763254938647151
Iteration 14/100 - Best Score: 0.000763254938647151
Iteration 15/100 - Best Score: 0.000763254938647151
Iteration 16/100 - Best Score: 0.000763254938647151
Iteration 17/100 - Best Score: 0.000763254938647151
Iteration 18/100 - Best Score: 0.000763254938647151
Iteration 19/100 - Best Score: 0.000763254938647151
Iteration 

In [9]:
# Train the final model using the optimized hyperparameters
lstm_units = int(best_hyperparameters[0])
learning_rate = best_hyperparameters[1]
batch_size = int(best_hyperparameters[2])

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(currency_data.values)

In [10]:
def create_dataset(data, time_step=60):
    X, y = [], []
    for i in range(time_step, len(data)):
        X.append(data[i-time_step:i, 0])  # past 'time_step' days
        y.append(data[i, 0])  # next day price
    return np.array(X), np.array(y)

X, y = create_dataset(scaled_data)

X = X.reshape(X.shape[0], X.shape[1], 1)

In [11]:
# Build the optimized LSTM model
model = Sequential()
model.add(tf.keras.layers.Input(shape=(X.shape[1], 1)))  # Corrected input layer
model.add(LSTM(units=lstm_units, return_sequences=True))
model.add(LSTM(units=lstm_units, return_sequences=False))
model.add(Dense(units=1))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mean_squared_error')

In [12]:
# Train the model
model.fit(X, y, epochs=10, batch_size=batch_size)

Epoch 1/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.0349
Epoch 2/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0020
Epoch 3/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0019
Epoch 4/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0015
Epoch 5/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0017
Epoch 6/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0022
Epoch 7/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0013
Epoch 8/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0013
Epoch 9/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0012
Epoch 10/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0016


In [13]:
# Save the trained model
model.save("optimized_lstm_currency_model.keras")
print("Model saved as 'optimized_lstm_currency_model.keras'")

Model saved as 'optimized_lstm_currency_model.keras'
